# TPOT tutorial on the Titanic dataset 

The Titanic machine learning competition on [Kaggle](https://www.kaggle.com/c/titanic) is one of the most popular beginner's competitions on the platform. We will use that competition here to demonstrate the implementation of TPOT. 

In [3]:
# Import required libraries
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

In [4]:
# Load the data
titanic = pd.read_csv('data/titanic_train.csv')
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Exploration 

In [3]:
titanic.groupby('Sex').Survived.value_counts()

Sex     Survived
female  1           233
        0            81
male    0           468
        1           109
Name: Survived, dtype: int64

In [4]:
titanic.groupby(['Pclass','Sex']).Survived.value_counts()

Pclass  Sex     Survived
1       female  1            91
                0             3
        male    0            77
                1            45
2       female  1            70
                0             6
        male    0            91
                1            17
3       female  0            72
                1            72
        male    0           300
                1            47
Name: Survived, dtype: int64

In [5]:
id = pd.crosstab([titanic.Pclass, titanic.Sex], titanic.Survived.astype(float))
id.div(id.sum(1).astype(float), 0)

Survived            0.0       1.0
Pclass Sex                       
1      female  0.031915  0.968085
       male    0.631148  0.368852
2      female  0.078947  0.921053
       male    0.842593  0.157407
3      female  0.500000  0.500000
       male    0.864553  0.135447

## Data Munging 

The first and most important step in using TPOT on any data set is to rename the target class/response variable to `class`.

In [6]:
titanic.rename(columns={'Survived': 'class'}, inplace=True)

At present, TPOT requires all the data to be in numerical format. As we can see below, our data set has 5 categorical variables which contain non-numerical values: `Name`, `Sex`, `Ticket`, `Cabin` and `Embarked`.

In [7]:
titanic.dtypes

PassengerId      int64
class            int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

We then check the number of levels that each of the five categorical variables have. 

In [8]:
for cat in ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']:
    print("Number of levels in category '{0}': \b {1:2.2f} ".format(cat, titanic[cat].unique().size))

Number of levels in category 'Name':  891.00 
Number of levels in category 'Sex':  2.00 
Number of levels in category 'Ticket':  681.00 
Number of levels in category 'Cabin':  148.00 
Number of levels in category 'Embarked':  4.00 


As we can see, `Sex` and `Embarked` have few levels. Let's find out what they are.

In [9]:
for cat in ['Sex', 'Embarked']:
    print("Levels for catgeory '{0}': {1}".format(cat, titanic[cat].unique()))

Levels for catgeory 'Sex': ['male' 'female']
Levels for catgeory 'Embarked': ['S' 'C' 'Q' nan]


We then code these levels manually into numerical values. For `nan` i.e. the missing values, we simply replace them with a placeholder value (-999). In fact, we perform this replacement for the entire data set.

In [10]:
titanic['Sex'] = titanic['Sex'].map({'male':0,'female':1})
titanic['Embarked'] = titanic['Embarked'].map({'S':0,'C':1,'Q':2})

In [11]:
titanic = titanic.fillna(-999)
pd.isnull(titanic).any()

PassengerId    False
class          False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool

Since `Name` and `Ticket` have so many levels, we drop them from our analysis for the sake of simplicity. For `Cabin`, we encode the levels as digits using Scikit-learn's [`MultiLabelBinarizer`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) and treat them as new features. 

In [12]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
CabinTrans = mlb.fit_transform([{str(val)} for val in titanic['Cabin'].values])

In [13]:
CabinTrans

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

Drop the unused features from the dataset. 

In [14]:
titanic_new = titanic.drop(['Name','Ticket','Cabin','class'], axis=1)

In [15]:
assert (len(titanic['Cabin'].unique()) == len(mlb.classes_)), "Not Equal" #check correct encoding done

We then add the encoded features to form the final dataset to be used with TPOT. 

In [16]:
titanic_new = np.hstack((titanic_new.values,CabinTrans))

In [17]:
np.isnan(titanic_new).any()

False

Keeping in mind that the final dataset is in the form of a numpy array, we can check the number of features in the final dataset as follows.

In [18]:
titanic_new[0].size

156

Finally we store the class labels, which we need to predict, in a separate variable. 

In [19]:
titanic_class = titanic['class'].values

## Data Analysis using TPOT

To begin our analysis, we need to divide our training data into training and validation sets. The validation set is just to give us an idea of the test set error. The model selection and tuning is entirely taken care of by TPOT, so if we want to, we can skip creating this validation set.

In [20]:
training_indices, validation_indices = training_indices, testing_indices = train_test_split(titanic.index, stratify = titanic_class, train_size=0.75, test_size=0.25)
training_indices.size, validation_indices.size

(668, 223)

After that, we proceed to calling the `fit`, `score` and `export` functions on our training dataset. To get a better idea of how these functions work, refer the TPOT documentation [here](http://epistasislab.github.io/tpot/api/).

An important TPOT parameter to set is the number of generations. Since our aim is to just illustrate the use of TPOT, we have set it to 5. On a standard laptop with 4GB RAM, it roughly takes 5 minutes per generation to run. For each added generation, it should take 5 mins more. Thus, for the default value of 100, total run time could be roughly around 8 hours.  

In [21]:
from config.classifier_models_only import classifier_config_dict
time_allocated = 60

In [22]:
tpot = TPOTClassifier(max_time_mins=time_allocated, config_dict=classifier_config_dict, verbosity=3)
tpot.fit(titanic_new[training_indices], titanic_class[training_indices])

11 operators have been imported by TPOT.


Optimization Progress: 100%|██████████| 100/100 [00:45<00:00,  1.83pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 Input X must be non-negative


Optimization Progress: 100%|██████████| 100/100 [00:45<00:00,  1.83pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 59
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='logistic_regression' are not supported when dual=True, Parameters: penalty='l1', loss='logistic_regression', dual=True


...

_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='squared_hinge' are not supported when dual=True, Parameters: penalty='l1', loss='squared_hinge', dual=True


Optimization Progress: 100%|██████████| 3400/3400 [59:06<00:00,  1.09pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 Input X must be non-negative



60.01370753333333 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT...0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]), 'min_child_weight': range(1, 21), 'nthread': [1]}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=1000000, max_eval_time_mins=5,
        max_time_mins=60, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=100, periodic_checkpoint_folder=None,
        population_size=100, random_state=None, scoring=None,
        subsample=1.0, verbosity=3, warm_start=False)

In [22]:
tpot.score(titanic_new[validation_indices], titanic.loc[validation_indices, 'class'].values)

0.8340807174887892

In [23]:
tpot.export('tpot_titanic_pipeline.py')

Let's have a look at the generated code. As we can see, the random forest classifier performed the best on the given dataset out of all the other models that TPOT currently evaluates on. If we ran TPOT for more generations, then the score should improve further.

In [ ]:
# %load tpot_titanic_pipeline.py
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the class is labeled 'class' in the data file
tpot_data = np.recfromcsv('PATH/TO/DATA/FILE', delimiter='COLUMN_SEPARATOR', dtype=np.float64)
features = np.delete(tpot_data.view(np.float64).reshape(tpot_data.size, -1), tpot_data.dtype.names.index('class'), axis=1)
training_features, testing_features, training_classes, testing_classes = \
    train_test_split(features, tpot_data['class'], random_state=42)

exported_pipeline = RandomForestClassifier(bootstrap=False, max_features=0.4, min_samples_leaf=1, min_samples_split=9)

exported_pipeline.fit(training_features, training_classes)
results = exported_pipeline.predict(testing_features)


### Make predictions on the submission data 

In [25]:
# Read in the submission dataset
titanic_sub = pd.read_csv('data/titanic_test.csv')
titanic_sub.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


The most important step here is to check for new levels in the categorical variables of the submission dataset that are absent in the training set. We identify them and set them to our placeholder value of '-999', i.e., we treat them as missing values. This ensures training consistency, as otherwise the model does not know what to do with the new levels in  the submission dataset. 

In [26]:
for var in ['Cabin']: #,'Name','Ticket']:
    new = list(set(titanic_sub[var]) - set(titanic[var]))
    titanic_sub.ix[titanic_sub[var].isin(new), var] = -999

We then carry out the data munging steps as done earlier for the training dataset.

In [27]:
titanic_sub['Sex'] = titanic_sub['Sex'].map({'male':0,'female':1})
titanic_sub['Embarked'] = titanic_sub['Embarked'].map({'S':0,'C':1,'Q':2})

In [28]:
titanic_sub = titanic_sub.fillna(-999)
pd.isnull(titanic_sub).any()

PassengerId    False
Pclass         False
Name           False
Sex            False
Age            False
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool

While calling `MultiLabelBinarizer` for the submission data set, we first fit on the training set again to learn the levels and then transform the submission dataset values. This further ensures that only those levels that were present in the training dataset are transformed. If new levels are still found in the submission dataset then it will return an error and we need to go back and check our earlier step of replacing new levels with the placeholder value. 

In [29]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
SubCabinTrans = mlb.fit([{str(val)} for val in titanic['Cabin'].values]).transform([{str(val)} for val in titanic_sub['Cabin'].values])
titanic_sub = titanic_sub.drop(['Name','Ticket','Cabin'], axis=1)

In [30]:
# Form the new submission data set
titanic_sub_new = np.hstack((titanic_sub.values,SubCabinTrans))

In [31]:
np.any(np.isnan(titanic_sub_new))

False

In [32]:
# Ensure equal number of features in both the final training and submission dataset
assert (titanic_new.shape[1] == titanic_sub_new.shape[1]), "Not Equal" 

In [33]:
# Generate the predictions
submission = tpot.predict(titanic_sub_new)

In [34]:
# Create the submission file
final = pd.DataFrame({'PassengerId': titanic_sub['PassengerId'], 'Survived': submission})
final.to_csv('data/submission.csv', index = False)

In [35]:
final.shape

(418, 2)

There we go! We have successfully generated the predictions for the 418 data points in the submission dataset, and we're good to go ahead to submit these predictions on Kaggle. 

# XGBOOST Experiment

In [23]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from src.hypersearch import xgbc_random
from config.xgboost import config_dict

In [24]:
xgbc_random(
    config_dict,
    3600,
    titanic_new[training_indices], 
    titanic_class[training_indices],
    titanic_new[validation_indices], 
    titanic.loc[validation_indices, 'class'].values)

Trace xgbc_random() --- starting new iteration --- time elapsed = 1.9073486328125e-06 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 1.0, 'subsample': 0.8500000000000001, 'min_child_weight': 3, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- best_score updated to 0.8251121076233184
Trace xgbc_random() --- best_kwargs updated to {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 1.0, 'subsample': 0.8500000000000001, 'min_child_weight': 3, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 0.1369800567626953 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.5, 'subsample': 0.9500000000000001, 'min_child_weight': 4, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 1.729741573333

Trace xgbc_random() --- starting new iteration --- time elapsed = 12.686507940292358 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 1, 'learning_rate': 0.1, 'subsample': 1.0, 'min_child_weight': 15, 'nthread': 1, 'gamma': 1.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 12.759617805480957 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 10, 'learning_rate': 1.0, 'subsample': 0.1, 'min_child_weight': 4, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 12.798751592636108 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.001, 'subsample': 0.9000000000000001, 'min_child_weight': 1, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 12.96160459518

Trace xgbc_random() --- starting new iteration --- time elapsed = 23.512521743774414 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 8, 'learning_rate': 0.01, 'subsample': 1.0, 'min_child_weight': 7, 'nthread': 1, 'gamma': 2.5, 'colsample_bytree': 0.7}
Trace xgbc_random() --- best_score updated to 0.8430493273542601
Trace xgbc_random() --- best_kwargs updated to {'n_estimators': 50, 'max_depth': 8, 'learning_rate': 0.01, 'subsample': 1.0, 'min_child_weight': 7, 'nthread': 1, 'gamma': 2.5, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 23.640545129776 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.5, 'subsample': 0.35000000000000003, 'min_child_weight': 9, 'nthread': 1, 'gamma': 5.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 23.70352268218994 seconds
Trace xgbc_random() --- trying hyperpa

Trace xgbc_random() --- starting new iteration --- time elapsed = 41.91838622093201 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 1, 'learning_rate': 1.0, 'subsample': 0.7000000000000001, 'min_child_weight': 17, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 42.303980350494385 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.9000000000000001, 'min_child_weight': 5, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 44.85118079185486 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 9, 'learning_rate': 0.001, 'subsample': 0.8500000000000001, 'min_child_weight': 8, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting 

Trace xgbc_random() --- starting new iteration --- time elapsed = 59.88252067565918 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.5, 'subsample': 0.1, 'min_child_weight': 8, 'nthread': 1, 'gamma': 2.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 59.920881032943726 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 10, 'learning_rate': 1.0, 'subsample': 0.1, 'min_child_weight': 5, 'nthread': 1, 'gamma': 1.5, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 59.96348452568054 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 5, 'learning_rate': 0.1, 'subsample': 0.15000000000000002, 'min_child_weight': 15, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 60.4802091121673

...

Trace xgbc_random() --- starting new iteration --- time elapsed = 3559.2865126132965 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.01, 'subsample': 0.8500000000000001, 'min_child_weight': 11, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3559.556977033615 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 0.01, 'subsample': 0.8, 'min_child_weight': 11, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3561.985985994339 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.5, 'subsample': 0.9000000000000001, 'min_child_weight': 7, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration -

Trace xgbc_random() --- starting new iteration --- time elapsed = 3576.946380853653 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 1, 'learning_rate': 0.01, 'subsample': 0.15000000000000002, 'min_child_weight': 10, 'nthread': 1, 'gamma': 0.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3577.15496635437 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 4, 'learning_rate': 0.5, 'subsample': 0.8500000000000001, 'min_child_weight': 4, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3577.2509186267853 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.5, 'subsample': 0.4, 'min_child_weight': 13, 'nthread': 1, 'gamma': 3.5, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3

Trace xgbc_random() --- starting new iteration --- time elapsed = 3587.7043175697327 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.001, 'subsample': 0.15000000000000002, 'min_child_weight': 7, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3587.7961192131042 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 8, 'learning_rate': 1.0, 'subsample': 0.7000000000000001, 'min_child_weight': 20, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3587.9313843250275 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 9, 'learning_rate': 1.0, 'subsample': 0.4, 'min_child_weight': 17, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration

Trace xgbc_random() --- starting new iteration --- time elapsed = 3599.6028950214386 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.5, 'subsample': 0.1, 'min_child_weight': 12, 'nthread': 1, 'gamma': 1.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3599.957364320755 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.5, 'subsample': 0.15000000000000002, 'min_child_weight': 11, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- duration exceeded --- process quitting with best_score = 0.852017937219731


({'n_estimators': 100,
  'max_depth': 9,
  'learning_rate': 0.5,
  'subsample': 0.4,
  'min_child_weight': 1,
  'nthread': 1,
  'gamma': 2.5,
  'colsample_bytree': 0.7999999999999999},
 0.852017937219731)

In [18]:
import imp
imp.reload(h2o)

<module 'h2o' from '/home/gunny/anaconda3/lib/python3.6/site-packages/h2o/__init__.py'>

In [1]:
import h2o
from h2o.automl import H2OAutoML

In [2]:
h2o.init(ip='192.168.1.191')

Checking whether there is an H2O instance running at http://192.168.1.191:54321. connected.


H2O cluster uptime:,5 mins 24 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,5 months and 6 days !!!
H2O cluster name:,gunny
H2O cluster total nodes:,1
H2O cluster free memory:,3.355 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [34]:
train = h2o.import_file("data/titanic_train.csv")
test = h2o.import_file("data/titanic_test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [35]:
train = train[:, ["PassengerId", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked", "Survived"]]

In [36]:
train['Sex'] = (train['Sex'] == 'male').ifelse(0, train["Sex"])
train["Sex"] = (train['Sex'] == 'female').ifelse(1, train["Sex"])
train["Embarked"] = (train['Embarked'] == 'S').ifelse(0, train["Embarked"])
train["Embarked"] = (train['Embarked'] == 'C').ifelse(1, train["Embarked"])
train["Embarked"] = (train['Embarked'] == 'Q').ifelse(2, train["Embarked"])

In [37]:
x = train.columns
y = "Survived"
x.remove(y)

In [38]:
train[y] = train[y].asfactor()

In [39]:
train["Sex"] = train["Sex"].asfactor()
train["Embarked"] = train["Embarked"].asfactor()
train["Pclass"] = train["Pclass"].asfactor()

In [40]:
aml = H2OAutoML(max_runtime_secs=3600)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
